# FireRoad053 Model

In [20]:
## if you have not installed package "CSV" or "JSON", please uncomment the line below and execute it
# using Pkg; Pkg.add("CSV")

using DataFrames, CSV
utility = CSV.read("data/utility_norm.csv", DataFrame)
classes = CSV.read("finaldata/parsedsp21_actual_classes.csv", DataFrame)

,Column1,id,total-units,prereqs,semester
,String,String,Int64,String?,String
1,14.01,14.01,12,missing,"['Fall', 'Spring']"
2,15.0251,15.0251,9,14.01,['Spring']
3,15.053,15.053,12,6.0001 or permission of instructor,['Spring']
4,15.068,15.068,9,missing,['Spring']
5,15.276,15.276,12,missing,"['Fall', 'Spring']"
6,15.501,15.501,12,missing,"['Fall', 'Spring']"
7,15.8141,15.8141,9,missing,['Spring']
8,18.01,18.01,12,missing,"['Fall', 'Spring']"
9,18.02,18.02,12,GIR:CAL1,"['Fall', 'Spring']"


# Model

In [21]:
#--- Model specification
using JuMP, DataFrames, Gurobi
M = 1000000
S = nrow(classes)
T = 8
min_units = 12
max_units = 60
#min_workload = 
#max_workload = 

model = Model(with_optimizer(Gurobi.Optimizer))

@variable(model, x[1:S, 1:T], Bin)

#@objective(model, Max, sum(utility[i, :] * sum(x[i,j] for j = 1:T) for i = 1:S))

Academic license - for non-commercial use only - expires 2021-07-07


50×8 Array{VariableRef,2}:
 x[1,1]   x[1,2]   x[1,3]   x[1,4]   x[1,5]   x[1,6]   x[1,7]   x[1,8]
 x[2,1]   x[2,2]   x[2,3]   x[2,4]   x[2,5]   x[2,6]   x[2,7]   x[2,8]
 x[3,1]   x[3,2]   x[3,3]   x[3,4]   x[3,5]   x[3,6]   x[3,7]   x[3,8]
 x[4,1]   x[4,2]   x[4,3]   x[4,4]   x[4,5]   x[4,6]   x[4,7]   x[4,8]
 x[5,1]   x[5,2]   x[5,3]   x[5,4]   x[5,5]   x[5,6]   x[5,7]   x[5,8]
 x[6,1]   x[6,2]   x[6,3]   x[6,4]   x[6,5]   x[6,6]   x[6,7]   x[6,8]
 x[7,1]   x[7,2]   x[7,3]   x[7,4]   x[7,5]   x[7,6]   x[7,7]   x[7,8]
 x[8,1]   x[8,2]   x[8,3]   x[8,4]   x[8,5]   x[8,6]   x[8,7]   x[8,8]
 x[9,1]   x[9,2]   x[9,3]   x[9,4]   x[9,5]   x[9,6]   x[9,7]   x[9,8]
 x[10,1]  x[10,2]  x[10,3]  x[10,4]  x[10,5]  x[10,6]  x[10,7]  x[10,8]
 x[11,1]  x[11,2]  x[11,3]  x[11,4]  x[11,5]  x[11,6]  x[11,7]  x[11,8]
 x[12,1]  x[12,2]  x[12,3]  x[12,4]  x[12,5]  x[12,6]  x[12,7]  x[12,8]
 x[13,1]  x[13,2]  x[13,3]  x[13,4]  x[13,5]  x[13,6]  x[13,7]  x[13,8]
 ⋮                                            

# Constraints

In [15]:
# 1. Each student takes (or gets credit for) all of the Science/Math GIRs.
# Math GIR
@constraint(model, math, sum(x[, t] for t = 1:T) == 1)

# Physics GIR
@constraint(model, physics, sum(x[, t] for t = 1:T) == 1)

# Chemistry GIR
@constraint(model, chem, sum(x[1, t] + x[2, t] for t = 1:T) == 1)

# Biology GIR
@constraint(model, bio, sum(x[23, t] + x[24, t] for t = 1:T) == 1)

# 2. Each student takes 8 HASS subjects.
@constraint(model, hass, sum( for t = 1:T) >= 8)

# 3. Each student satisfies the requirements of her major (and minor).


# 4. No subject is taken prior to its prerequisites.


# 5. Each student takes 216 units beyond GIRs.
@constraint(model, units, sum(x[s, t] * classes[s, 3] for s = 1:S for t = 1:T) >= 210)

# 6. No two subjects taken in Spring 2021 should overlap in time.

    
# 7. Students cannot take the same subject twice.
@constraint(model, once, sum(x[s, t] for s = 1:S for t = 1:T) <= 1)

# 8. Each student can specify if they want to take a specific subject in the future.
@constraint(model, req[s in 1:S], sum(x[s, t] for t in 1:T) >= 1 - M * (1 - REQ CSV[s]))

# 9. Each student can specify which semester they want to take a subject.
@constraint(model, reqsem[s in 1:S, t in 1:T], x[s, t] >= 1 - M * (1 - REQSEM CSV[s, t]))

# 10. Each student can specify which semester they do not want to take a subject. 
@constraint(model, reqnot[s in 1:S, t in 1:T], x[s, t] <= 1 + M * (1 - REQNOT CSV[s, t]))
    
# 11. The minimum and maximum number of units that each student wants to take each semester.
@constraint(model, minunits[t in 1:T], sum(x[s, t] * classes[s, 3] for s in 1:S) >= min_units)
@constraint(model, maxunits[t in 1:T], sum(x[s, t] * classes[s, 3] for s in 1:S) <= max_units)
    
# 12. The minimum and maximum workload (in number of hours) that each student wants to take on each semester.
@constraint(model, minwork[t in 1:T], sum(x[s, t] * classes[s, WORKLOAD COL] for s in 1:S) >= min_workload)
@constraint(model, maxwork[t in 1:T], sum(x[s, t] * classes[s, WORKLOAD COL] for s in 1:S) <= max_workload)

8-element Array{ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where C,1}:
 maxunits[1] : 12 x[1,1] + 12 x[2,1] + 12 x[3,1] + 12 x[4,1] + 6 x[5,1] + 6 x[6,1] + 6 x[7,1] + 6 x[8,1] + 12 x[9,1] + 12 x[10,1] + 12 x[11,1] + 12 x[12,1] + 12 x[13,1] + 15 x[14,1] + 12 x[15,1] + 12 x[16,1] + 12 x[17,1] + 12 x[18,1] + 12 x[19,1] + 12 x[20,1] + 12 x[21,1] + 9 x[22,1] + 12 x[23,1] + 12 x[24,1] + 12 x[25,1] + 12 x[26,1] + 12 x[27,1] + 9 x[28,1] + 12 x[29,1] + 12 x[30,1] + 12 x[31,1] + 9 x[32,1] + 12 x[33,1] + 12 x[34,1] + 12 x[35,1] + 12 x[36,1] + 12 x[37,1] + 9 x[38,1] + 12 x[39,1] + 12 x[40,1] + 12 x[41,1] + 12 x[42,1] + 12 x[43,1] + 12 x[44,1] ≤ 60.0
 maxunits[2] : 12 x[1,2] + 12 x[2,2] + 12 x[3,2] + 12 x[4,2] + 6 x[5,2] + 6 x[6,2] + 6 x[7,2] + 6 x[8,2] + 12 x[9,2] + 12 x[10,2] + 12 x[11,2] + 12 x[12,2] + 12 x[13,2] + 15 x[14,2] + 12 x[15,2] + 12 x[16,2] + 12 x[17,2] + 12 x[18,2] + 12 x[19,2] + 12 x[20,2] + 12 x[21,2] + 9 x[22,2] + 12 x[23,2] + 12 x[24,2] + 12 x[25,2] + 12 x[26,2] + 12

In [ ]:
#--- Write codes here to print your solutions
optimize!(model)
@show termination_status(model)
@show objective_value(model)

In [ ]:
display("text/csv", value.(x)) # could also display JSON